In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 499.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.2 MB/s eta 0:00:00


In [ ]:
%pip install -qU langchain_mistralai

In [ ]:
import getpass
import os
os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter your Mistral API key: ")

Enter your Mistral API key: ··········


In [ ]:
from langchain.agents import initialize_agent, AgentExecutor
from langchain_mistralai import ChatMistralAI
from langchain.tools import Tool
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
# Agent Logic Functions
def solution_focused_tool(input_text, debating_history):
    # Logic for solution-focused responses considering the debating history
    return f"Let's focus on your goals and find ways to achieve them. {input_text}"

def reframing_tool(input_text, debating_history):
    # Logic for reframing responses considering the debating history
    return f"Let's find a new perspective that highlights your strengths and opportunities. {input_text}"

def positive_regard_tool(input_text, debating_history):
    # Logic for positive regard responses considering the debating history
    return f"I'm here to support you fully, without judgment, no matter what you're facing. {input_text}"

In [ ]:
solution_focused = Tool(
    name="Solution-Focused",
    func=solution_focused_tool,
    description="Provides goal-oriented solutions to problems."
)

reframing = Tool(
    name="Reframing",
    func=reframing_tool,
    description="Helps users view situations in a more positive or constructive way."
)

positive_regard = Tool(
    name="Positive-Regard",
    func=positive_regard_tool,
    description="Offers non-judgmental and empathetic support."
)

In [ ]:
llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    max_retries=2,
)

In [ ]:
def multi_agent_debating(input_text, num_turns=3):
    debating_history = []  # To store the responses from each agent during each turn

    for turn in range(num_turns):
        turn_responses = {}
        # Each agent generates a response based on the current history
        turn_responses['Solution-Focused'] = solution_focused_tool(input_text, debating_history)
        turn_responses['Reframing'] = reframing_tool(input_text, debating_history)
        turn_responses['Positive-Regard'] = positive_regard_tool(input_text, debating_history)

        # Add responses to the history and refine scores or weightings based on evaluations
        debating_history.append(turn_responses)

    return debating_history

In [ ]:
# Calculate Attribute Scores
def calculate_attribute_scores(debating_history):
    scores = {'Solution-Focused': 0, 'Reframing': 0, 'Positive-Regard': 0}

    for turn in debating_history:
        for attribute, response in turn.items():
            # Example scoring logic: Assign scores based on keyword detection or sentiment
            if "goal" in response or "action" in response:
                scores['Solution-Focused'] += 1
            if "new perspective" in response or "positive change" in response:
                scores['Reframing'] += 1
            if "support" in response or "no judgment" in response:
                scores['Positive-Regard'] += 1

    # Normalize scores to create weights
    total = sum(scores.values())
    for attribute in scores:
        scores[attribute] /= total

    return scores

In [ ]:
# Create a Tailored Counselor Profile
def tailored_counselor(input_text):
    debating_history = multi_agent_debating(input_text, num_turns=3)
    attribute_scores = calculate_attribute_scores(debating_history)

    # Return the counselor profile with attribute weights
    return {
        "Solution-Focused": debating_history[-1]['Solution-Focused'],  # Latest response
        "Reframing": debating_history[-1]['Reframing'],
        "Positive-Regard": debating_history[-1]['Positive-Regard'],
        "Scores": attribute_scores
    }

In [ ]:
# Define a Prompt Template for Response Generation
response_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a compassionate and insightful counselor."),
        (
            "user",
            "The user expressed: '{input_text}'\n"
            "Counselor Profile Summary:\n"
            "Solution-Focused (Weight: {solution_score}): {solution_focused}\n"
            "Reframing (Weight: {reframing_score}): {reframing}\n"
            "Positive-Regard (Weight: {regard_score}): {positive_regard}\n"
            "Provide a thoughtful and empathetic response based on the given counselor profile."
        ),
    ]
)

In [ ]:
# Generate Final Response
def generate_response(input_text, counselor_profile):
    # Format the prompt using the counselor profile data
    formatted_prompt = response_prompt_template.format_messages(
        input_text=input_text,
        solution_focused=counselor_profile["Solution-Focused"],
        reframing=counselor_profile["Reframing"],
        positive_regard=counselor_profile["Positive-Regard"],
        solution_score=counselor_profile["Scores"]['Solution-Focused'],
        reframing_score=counselor_profile["Scores"]['Reframing'],
        regard_score=counselor_profile["Scores"]['Positive-Regard'],
    )

    # Send the formatted prompt to the LLM
    final_response = llm(formatted_prompt)
    return final_response

In [ ]:
input_text = "Well u wanna talk?"
counselor_profile = tailored_counselor(input_text)
final_response = generate_response(input_text, counselor_profile)

print(final_response.content)

content="Of course, I'm here for you. It sounds like you might have something on your mind. I'm really glad you reached out. Let's talk about whatever you'd like, and we can explore it together. I'm here to support you, without any judgment, and maybe we can even find a new perspective or some goals to work towards. What's been going on?" additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 200, 'total_tokens': 283, 'completion_tokens': 83}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'} id='run-2b2d29fe-3220-4f7a-9881-93a24f9eb2bc-0' usage_metadata={'input_tokens': 200, 'output_tokens': 83, 'total_tokens': 283}
